In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
# Download the file
path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

path_to_file = os.path.dirname(path_to_zip)+"/spa-eng/spa.txt"

# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

en_sentence = u"May I borrow this book?"
sp_sentence = u"¿Puedo tomar prestado este libro?"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence).encode('utf-8'))

# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, SPANISH]
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]

  return zip(*word_pairs)

en, sp = create_dataset(path_to_file, None)
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
  targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer


# Try experimenting with the size of that dataset
num_examples = 30000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

2646016/2638744 [==============================] - 0s 0us/step
<start> may i borrow this book ? <end>
b'<start> \xc2\xbf puedo tomar prestado este libro ? <end>'
24000 24000 6000 6000


In [2]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256 # embedding size
units = 1024 # size of hidden state
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [3]:
# class Encoder(tf.keras.Model):
#   def __init__(self, embedding_dim, units):
#     super(Encoder, self).__init__()
#     self.embedding = tf.keras.layers.Embedding(vocab_inp_size, embedding_dim)
#     self.lstm = tf.keras.layers.LSTM(units, return_state=True)

#   def call(self, input_sentence):
#     embedding_input_sentence = self.embedding(input_sentence)
#     output, hidden_h, hidden_c = self.lstm(embedding_input_sentence)
#     return [hidden_h, hidden_c]

class Encoder(tf.keras.Model):
  def __init__(self, embedding_dim, units):
    super(Encoder, self).__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_inp_size, embedding_dim)
    self.lstm = tf.keras.layers.LSTM(units, return_state=True)
  def call(self, input_sentence):
    # embedding_input_sentence shape: (batch_size, max_input_length, embedding_dim)
    embedding_input_sentence = self.embedding(input_sentence)
    # hidden_h shape: (batch_size, units)
    # hidden_c shape: (batch_size, units)
    output, hidden_h, hidden_c = self.lstm(embedding_input_sentence)
    return [hidden_h, hidden_c]

In [4]:
encoder = Encoder(embedding_dim, units)
input = tf.zeros((BATCH_SIZE, max_length_inp))
# encoder(input)[0].shape, encoder(input)[1].shape
thought_vector = encoder(input)

In [6]:
# class Decoder(tf.keras.Model):
#   def __init__(self, embedding_dim, units):
#     super(Decoder, self).__init__()
#     self.embedding = tf.keras.layers.Embedding(vocab_tar_size, embedding_dim)
#     self.lstm = tf.keras.layers.LSTM(units, return_state=True)
#     self.fc = tf.keras.layers.Dense(units)

#   def call(self, x, last_state):
#     embedding_x = self.embedding(x)
#     output, hidden_h, hidden_c = self.lstm(embedding_x, initial_state=last_state)
#     return self.fc(hidden_h), [hidden_h, hidden_c]

class Decoder(tf.keras.Model):
  def __init__(self, embedding_dim, units):
    super(Decoder, self).__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_tar_size, embedding_dim)
    self.lstm = tf.keras.layers.LSTM(units, return_state=True)
    self.fc = tf.keras.layers.Dense(vocab_tar_size)
  def call(self, x, last_state):
    # x shape: (BATCH_SIZE, 1)
    x_embedding = self.embedding(x)
    output, hidden_h, hidden_c = self.lstm(x_embedding, initial_state=last_state)
    # (BATCH_SIZE, vocab_tar_size)
    return self.fc(hidden_h), [hidden_h, hidden_c]

decoder = Decoder(embedding_dim, units)
decoder_input = tf.zeros((BATCH_SIZE, 1))
output, updated_thought_vector = decoder(decoder_input, thought_vector)

In [7]:
loss_ob = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def cal_loss(y_true, y_pred):
  loss = loss_ob(y_true, y_pred)
  print(loss)
  mask = tf.cast(y_true != 0, dtype=loss.dtype)
  return tf.reduce_mean(loss * mask)

In [26]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam()

In [27]:
@tf.function
def train_step(source_sentence, target_sentence):
  #
  # target_sentence shape: (BATCH_SZIE, target_sentence_length)
  loss = 0
  with tf.GradientTape() as tape:  # compute gradient
    encode_state = encoder(source_sentence)
    # BATCH_SIZE, 1
    input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)
    current_state = encode_state

    for i in range(1, target_sentence.shape[1]):
      # output_decoder shape: (BATCH_SZIE, TARGET_VOCAB_SIZE)
      output_decoder, current_state = decoder(input, current_state)
      # print('output_decoder shape', output_decoder.shape)
      loss += cal_loss(target_sentence[:, i], output_decoder)
      # teacher forcing
      input = target_sentence[:, i:i+1]
  
  batch_loss = loss / int(target_sentence.shape[1])
  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return batch_loss

In [28]:
EPOCHS = 25

for i in range(EPOCHS):
  total_loss = 0
  for (batch, (source_sentence, target_sentence)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(source_sentence, target_sentence)
    total_loss += batch_loss

    if batch % 100 == 0:
      print("Batch loss is {}".format(batch_loss))
    
  if i % 10 == 0:
    print("Epoch: {}. Total loss is {}".format(i, total_loss/steps_per_epoch))

Tensor("sparse_categorical_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("sparse_categorical_crossentropy_1/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("sparse_categorical_crossentropy_2/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("sparse_categorical_crossentropy_3/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("sparse_categorical_crossentropy_4/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("sparse_categorical_crossentropy_5/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("sparse_categorical_crossentropy_6/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("sparse_categorical_crossentropy_7/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("sparse_categorical_crossentropy_8/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("sparse_categorical_crossentropy_9/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("sparse_categorical_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Ten

KeyboardInterrupt: ignored

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)
  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)
    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))
    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)
    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)
    return context_vector, attention_weights